In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
import math
from torch import Tensor
from typing import List, Optional
from torch.optim import Optimizer

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels

In [40]:
#Function to do updates according to Adam's Formula
def adam(params: List[Tensor],grads: List[Tensor],exp_avgs: List[Tensor],exp_avg_sqs: List[Tensor],
         max_exp_avg_sqs: List[Tensor],state_steps: List[int],*,beta1: float,beta2: float,lr: float,
         eps: float):
    
    for i, param in enumerate(params):

        grad = grads[i]
        exp_avg = exp_avgs[i]
        exp_avg_sq = exp_avg_sqs[i]
        step = state_steps[i]

        bias_correction1 = 1 - beta1 ** step
        bias_correction2 = 1 - beta2 ** step

        #Configuring the first and second moment running average coefficient
        exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
        exp_avg_sq.mul_(beta2).addcmul_(grad, grad.conj(), value=1 - beta2)
        
        denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(eps)

        step_size = lr / bias_correction1

        param.addcdiv_(exp_avg, denom, value=-step_size)

#Creating Adam Optimizer which overrides the step function.
class Adam(Optimizer):

    #Constructor with 4 parameters: LR, Beta rate and epsilon
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-7):

        defaults = dict(lr=lr, betas=betas, eps=eps)
        
        #Calling constructor of in-built PyTorch
        super(Adam, self).__init__(params, defaults)

    #THIS IS THE PART WHERE WE ARE IMPLEMENTING THE ADAM OPTIMIZER. 
    @torch.no_grad()
    def step(self):
        
        loss = None

        for group in self.param_groups:
            #Creating list to store various model parameters
            params_with_grad = []
            grads = []
            exp_avgs = []
            exp_avg_sqs = []
            max_exp_avg_sqs = []
            state_steps = []
            beta1, beta2 = group['betas']

            for p in group['params']:
                if p.grad is not None:
                    params_with_grad.append(p)
                    grads.append(p.grad)

                    state = self.state[p]
                    if len(state) == 0:
                        state['step'] = 0
                        #Gradient values' exponential moving average
                        state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        
                        #Squared gradient values' exponential moving average
                        state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                       
                    exp_avgs.append(state['exp_avg'])
                    exp_avg_sqs.append(state['exp_avg_sq'])

                    #Updating the number of steps for each param group
                    state['step'] += 1
                    
                    #Store and append the step after update
                    state_steps.append(state['step'])

            #Calling Adam's function to update parameters
            adam(params_with_grad, grads, exp_avgs,exp_avg_sqs,max_exp_avg_sqs,state_steps,
                   beta1=beta1,beta2=beta2,lr=group['lr'],eps=group['eps'])
            
        #return value of loss   
        return loss

In [41]:
#Class to define FL parameters
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 5
        self.rounds = 100
        self.epochs = 5
        self.local_batches = 128
        self.lr = 0.01
        self.C = 0.9
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

# use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [42]:
## creating clients for FL

hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [43]:
# Download FashionMNIST manually using 'wget' then uncompress the file
!wget www.di.ens.fr/~lelarge/FashionMNIST.tar.gz
!tar -zxvf FashionMNIST.tar.gz

zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'FashionMNIST.tar.gz'


In [44]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [45]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
    client['samples'] = len(trainset_ind_list) / args.images

In [46]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = datasets.FashionMNIST('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

global_train_dataset = datasets.FashionMNIST('./', train=True, download=True, transform=transform)
global_train_loader = DataLoader(global_train_dataset, batch_size=args.local_batches, shuffle=True)

In [47]:
## creating neural net structure using pytorch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [48]:
## function to do updates during FL model rounds (calls step function)

def ClientUpdate(args, device, client):
    client['model'].train()
    client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
            data = data.send(client['hook'])
            target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0:
                loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
    client['model'].get() 

In [49]:
## prints values periodically on testing data
def test(args, model, device, test_loader, name, isPrint):

    model.eval()   
    test_losss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_losss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_losss /= len(test_loader.dataset)
    
    if isPrint:
        print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_losss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        test_loss.append(test_losss)
        test_acc.append(correct/len(test_loader.dataset))
        
    else:
        train_loss.append(test_losss)
        train_acc.append(correct/len(test_loader.dataset))
   
        
    #return loss_list

In [54]:
torch.manual_seed(args.torch_seed)
global_model = Net()

train_loss = []
test_loss = []
train_acc = []
test_acc = []

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = Adam(client['model'].parameters())
    
for fed_round in range(args.rounds):
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
        
    # Averaging 
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global',True)
    test(args, global_model, device, global_train_loader, 'Global',False)
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

Model client2 Train Epoch: 1 [0/12032 (0%)]	Loss: 2.303589
Model client2 Train Epoch: 1 [1280/12032 (11%)]	Loss: 0.971004
Model client2 Train Epoch: 1 [2560/12032 (21%)]	Loss: 0.765004
Model client2 Train Epoch: 1 [3840/12032 (32%)]	Loss: 0.761951
Model client2 Train Epoch: 1 [5120/12032 (43%)]	Loss: 0.653680
Model client2 Train Epoch: 1 [6400/12032 (53%)]	Loss: 0.787424
Model client2 Train Epoch: 1 [7680/12032 (64%)]	Loss: 0.591428
Model client2 Train Epoch: 1 [8960/12032 (74%)]	Loss: 0.712314
Model client2 Train Epoch: 1 [10240/12032 (85%)]	Loss: 0.703854
Model client2 Train Epoch: 1 [11520/12032 (96%)]	Loss: 0.633102
Model client4 Train Epoch: 1 [0/12032 (0%)]	Loss: 2.324175
Model client4 Train Epoch: 1 [1280/12032 (11%)]	Loss: 1.173134
Model client4 Train Epoch: 1 [2560/12032 (21%)]	Loss: 1.042286
Model client4 Train Epoch: 1 [3840/12032 (32%)]	Loss: 0.697876
Model client4 Train Epoch: 1 [5120/12032 (43%)]	Loss: 0.774610
Model client4 Train Epoch: 1 [6400/12032 (53%)]	Loss: 0.62643

In [55]:
train_loss, test_loss

([1.581006516265869, 1.500590850830078],
 [1.5870809768676757, 1.5072730680465698])

In [58]:
print(np.multiply(test_acc,100))

[79.07 81.34]


In [ ]:
x = [(i+1) for i in range(len(train_loss))]

In [ ]:
import matplotlib.pyplot as plt

#plt.plot(x,train_loss)
#plt.plot(x,test_loss)
plt.plot(x,train_acc)
plt.plot(x,test_acc)
plt.show()